In [7]:
import pandas as pd
import re
import os
import ast
import json
def contains_ec_regex(s):
    return bool(re.search(r'_ec', s))



dataset = 'assays'
type_d = 'Joinable'
directory_path = f'data/val-exp/{dataset}/{type_d}'

if not os.path.exists(directory_path):
    raise FileNotFoundError(f"The path '{directory_path}' does not exist.")
        
# Print the immediate subfolders
print(f"Immediate subfolders of '{directory_path}':")
my_list = os.listdir(directory_path)
cnt = 0
for file in my_list:
    print(f"------ {cnt}/{len(my_list)} ------------ {dataset} {type_d} \n")
    if contains_ec_regex(file):
        cnt += 1
        continue
    
    file_path = f'{directory_path}/{file}'

    df_source = pd.read_csv(f"{file_path}/pyjedai/{file}_source.csv")
    df_target = pd.read_csv(f"{file_path}/pyjedai/{file}_target.csv")
    gtruth_filename = f'{file_path}/{file.lower()}_mapping.json'
    
    #read the json file
    with open(gtruth_filename, 'r') as f:
        gtruth_data = json.load(f)
    
    matching_dict = {}

    for pair in gtruth_data['matches']:
        matching_dict[pair['source_column']] = pair['target_column']

    test_data = []
    print(df_target)
    offset = df_source.shape[0]
    for _, row in df_source.iterrows():
        source_str = f"[ATT] {row['id']}"
        source_val = ""

        if isinstance(row['data'], str):
            data_list = ast.literal_eval(row['data'])
            val_list = " [VAL] ".join(data_list)
            source_val = f" [VAL] {val_list}"


        source_str += source_val
        
        for _, t_row in df_target.iterrows():
            target_str = f"[ATT] {t_row['id'] + offset}"
            target_val = ""
            if isinstance(t_row['data'], str):
                t_data_list = ast.literal_eval(t_row['data'])
                t_val_list = " [VAL] ".join(t_data_list)
                target_val = f" [VAL] {t_val_list}"
            target_str += target_val
            matching = 0
            if row['attributes'] in matching_dict and matching_dict[row['attributes']] == t_row['attributes']:
                matching = 1
            test_data.append([source_str, target_str, matching])    
            print(source_str)
            print(target_str)
            print(matching)
            break
        break
    break

Immediate subfolders of 'data/val-exp/assays/Joinable':
------ 0/48 ------------ assays Joinable 

    id     attributes                                               data
0    0     assays_ASI  ['637', '5210', '5222', '5221', '5220', '522',...
1    1   assays_CURAB                 ['intermedi', 'autocur', 'expert']
2    2      assays_Ti  ['50597', '22224', '50588', '50594', '10576', ...
3    3   assays_ASSTI  ['plasma', 'blood', 'urin', 'liver', 'kidney',...
4    4   assays_ASSCA                                                NaN
5    5   assays_Descr  ['rat', 'mg/kg', 'compound', 'receptor', 'dose...
6    6  assays_RELATT                                    ['n', 'h', 'u']
7    7   assays_CONSC  ['1', '8', '0', '9', '4', '6', '5', '7', '2', ...
8    8     assays_SRI                                              ['1']
9    9   assays_SASSI                                                NaN
10  10     assays_CHI  ['chembl626252', 'chembl624812', 'chembl624825...
11  11      assays_CI  ['